In [1]:
import gensim
import os
from distutils.version import LooseVersion, StrictVersion
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import re

In [2]:
from gensim.models import Word2Vec

In [3]:
modelwv = Word2Vec.load("W2Vmodel.bin")

In [4]:
# print(modelwv.wv['việt_nam'])

In [5]:
# modelwv.wv.most_similar('việt_nam', topn=5)

In [6]:
len(modelwv.wv)

137183

In [7]:
data = pd.read_csv('data_after.csv')

In [8]:
data.head()

,Unnamed: 0,text,lable,word_length
0,0,cháy căn_hộ nhà_tầng sài_gòn lửa cháy dữ_dội c...,Chinhtrixahoi,179
1,1,chủ xe cá_nhân có_thể xác_nhận điện_tử giãn ch...,Chinhtrixahoi,283
2,2,xây bãi xe_ngầm tầng sân golf trung_tâm đà lạt...,Chinhtrixahoi,207
3,3,bộ_đội việt_nam phát_hiện vị_trí dấu_hiệu sống...,Chinhtrixahoi,260
4,4,đường võ chí công hoàn_thành xe_khách chạy quả...,Chinhtrixahoi,527


In [2]:
from pyvi import ViTokenizer

In [ ]:
vocab_size = 300

In [9]:
len(modelwv.wv)

137183

In [10]:
vocab=list(modelwv.wv.index_to_key)

In [11]:
word_vec_dict={}
for word in vocab:
  word_vec_dict[word]=modelwv.wv[word]
print("The no of key-value pairs : ",len(word_vec_dict))

The no of key-value pairs :  137183


In [12]:
maxi=-1
for i,rev in enumerate(data['text']):
    tokens=rev.split()
    if(len(tokens)>maxi):
        maxi=len(tokens)
print(maxi)

3677


In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
lable_types = ('Chinhtrixahoi','Doisong','Khoahoc','Kinhdoanh','Phapluat','Suckhoe','Thegioi','Thethao','Giaitri','Sohoa')
lable_df = pd.DataFrame(lable_types, columns=['lable'])
labelencoder = LabelEncoder()
data['new_lable'] = labelencoder.fit_transform(data['lable'])
data

,Unnamed: 0,text,lable,word_length,new_lable
0,0,cháy căn_hộ nhà_tầng sài_gòn lửa cháy dữ_dội c...,Chinhtrixahoi,179,0
1,1,chủ xe cá_nhân có_thể xác_nhận điện_tử giãn ch...,Chinhtrixahoi,283,0
2,2,xây bãi xe_ngầm tầng sân golf trung_tâm đà lạt...,Chinhtrixahoi,207,0
3,3,bộ_đội việt_nam phát_hiện vị_trí dấu_hiệu sống...,Chinhtrixahoi,260,0
4,4,đường võ chí công hoàn_thành xe_khách chạy quả...,Chinhtrixahoi,527,0
...,...,...,...,...,...
44605,48893,hlv srimaka thái lan đối_thủ sea games hlv wor...,Thethao,194,9
44606,48894,cụ nước_rút ngoạn_mục vô_địch thế_giới lanvđv ...,Thethao,187,9
44607,48895,cầu_thủ indonesia khóc quyền dự world_cup indo...,Thethao,239,9
44608,48896,alcaraz_đấu sinner bán_kết miami open mỹcarlos...,Thethao,191,9


In [14]:
import keras
from keras.preprocessing.text import one_hot,Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split,cross_validate
from keras.models import Sequential
from keras.layers import Dense , Flatten ,Embedding,Input,CuDNNLSTM,LSTM
from keras.models import Model
from keras.preprocessing.text import text_to_word_sequence

In [ ]:
tok = Tokenizer()
tok.fit_on_texts(data['text'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(data['text'])

In [ ]:
vocab_size

In [ ]:
max_rev_len=3677  # max lenght of a review
vocab_size = len(tok.word_index) + 1  # total no of words
embed_dim=300 # embedding dimension as choosen in word2vec constructor

In [ ]:
pad_rev= pad_sequences(encd_rev, maxlen=max_rev_len, padding='post')
pad_rev.shape 

In [ ]:
pad_rev[0]

In [ ]:
embed_matrix=np.zeros(shape=(vocab_size,embed_dim))
for word,i in tok.word_index.items():
  embed_vector=word_vec_dict.get(word)
  if embed_vector is not None:  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

In [ ]:
embed_matrix.shape

In [ ]:
Y=keras.utils.to_categorical(data['new_lable'])
x_train,x_test,y_train,y_test=train_test_split(pad_rev,Y,test_size=0.20,random_state=42)

In [ ]:
Y.shape

In [ ]:
from keras.initializers import Constant
from keras.layers import ReLU
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM, Bidirectional

In [ ]:

model=Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embed_dim,input_length=max_rev_len,embeddings_initializer=Constant(embed_matrix)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(64, return_sequences=True, input_shape=(max_rev_len, ))))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.optimizer.learning_rate

In [ ]:
epochs=10
batch_size=64

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
history=model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size,validation_data=0.2,callbacks=[EarlyStopping(monitor='val_loss',patience=7, min_delta=0.0001)])

   2/1116 [..............................] - ETA: 8:42:49 - loss: 2.3672 - accuracy: 0.1875 

In [ ]:
model

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from keras.models import load_model

In [ ]:
model.evaluate(x_test,y_test)